In [8]:
!pip install pandas numpy matplotlib seaborn scikit-learn tkcalendar

   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.2 MB 3.0 MB/s eta 0:00:04
   ---- ----------------------------------- 1.0/10.2 MB 3.1 MB/s eta 0:00:03
   ------ --------------------------------- 1.6/10.2 MB 2.6 MB/s eta 0:00:04
   -------- ------------------------------- 2.1/10.2 MB 2.7 MB/s eta 0:00:03
   ----------- ---------------------------- 2.9/10.2 MB 2.8 MB/s eta 0:00:03
   ------------- -------------------------- 3.4/10.2 MB 2.9 MB/s eta 0:00:03
   ---------------- ----------------------- 4.2/10.2 MB 2.9 MB/s eta 0:00:03
   ------------------ --------------------- 4.7/10.2 MB 2.9 MB/s eta 0:00:02
   -------------------- ------------------- 5.2/10.2 MB 2.9 MB/s eta 0:00:02
   ---------------------- ----------------- 5.8/10.2 MB 2.9 MB/s eta 0:00:02
   ------------------------ --------------- 6.3/10.2 MB 2.9 MB/s eta 0:00:02
   --------------------------- ------------ 7.1/10.2 MB 2.9 MB/s eta 0:00:02
   ---


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import tkinter as tk
from tkinter import ttk, messagebox
from tkcalendar import DateEntry
import sqlite3
import csv
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

class ExpenseTracker:
    def __init__(self, root):
        self.root = root
        self.root.title("Expense Tracker with Prediction")
        self.root.geometry("800x600")
        
        # Initialize database
        self.init_database()
        
        # Create GUI
        self.create_widgets()
        
        # Load expenses
        self.load_expenses()
    
    def init_database(self):
        """Database Connectivity - Create SQLite database and table"""
        self.conn = sqlite3.connect('expenses.db')
        self.cursor = self.conn.cursor()
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS expenses (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                date TEXT,
                category TEXT,
                amount REAL,
                description TEXT
            )
        ''')
        self.conn.commit()
    
    def create_widgets(self):
        """Tkinter - Create GUI elements"""
        # Title
        title = tk.Label(self.root, text="Expense Tracker", font=("Arial", 20, "bold"), fg="#2c3e50")
        title.pack(pady=10)
        
        # Input Frame
        input_frame = tk.Frame(self.root, bg="#ecf0f1")
        input_frame.pack(pady=10, padx=20, fill="x")
        
        # Date
        tk.Label(input_frame, text="Date:", bg="#ecf0f1").grid(row=0, column=0, padx=5, pady=5)
        self.date_entry = DateEntry(input_frame, width=12, background='darkblue',
                                    foreground='white', borderwidth=2, date_pattern='yyyy-mm-dd')
        self.date_entry.grid(row=0, column=1, padx=5, pady=5)
        
        # Category
        tk.Label(input_frame, text="Category:", bg="#ecf0f1").grid(row=0, column=2, padx=5, pady=5)
        self.category_var = tk.StringVar()
        self.category_combo = ttk.Combobox(input_frame, textvariable=self.category_var, width=15)
        self.category_combo['values'] = ('Food', 'Transport', 'Entertainment', 'Bills', 'Shopping', 'Other')
        self.category_combo.grid(row=0, column=3, padx=5, pady=5)
        
        # Amount
        tk.Label(input_frame, text="Amount:", bg="#ecf0f1").grid(row=1, column=0, padx=5, pady=5)
        self.amount_entry = tk.Entry(input_frame, width=15)
        self.amount_entry.grid(row=1, column=1, padx=5, pady=5)
        
        # Description
        tk.Label(input_frame, text="Description:", bg="#ecf0f1").grid(row=1, column=2, padx=5, pady=5)
        self.desc_entry = tk.Entry(input_frame, width=30)
        self.desc_entry.grid(row=1, column=3, padx=5, pady=5)
        
        # Buttons Frame
        btn_frame = tk.Frame(self.root)
        btn_frame.pack(pady=10)
        
        tk.Button(btn_frame, text="Add Expense", command=self.add_expense, bg="#27ae60", fg="white", width=12).grid(row=0, column=0, padx=5)
        tk.Button(btn_frame, text="View Chart", command=self.show_chart, bg="#3498db", fg="white", width=12).grid(row=0, column=1, padx=5)
        tk.Button(btn_frame, text="Predict Next Month", command=self.predict_expense, bg="#e74c3c", fg="white", width=15).grid(row=0, column=2, padx=5)
        tk.Button(btn_frame, text="Export to CSV", command=self.export_csv, bg="#f39c12", fg="white", width=12).grid(row=0, column=3, padx=5)
        
        # Treeview for displaying expenses
        tree_frame = tk.Frame(self.root)
        tree_frame.pack(pady=10, padx=20, fill="both", expand=True)
        
        self.tree = ttk.Treeview(tree_frame, columns=("Date", "Category", "Amount", "Description"), show="headings")
        self.tree.heading("Date", text="Date")
        self.tree.heading("Category", text="Category")
        self.tree.heading("Amount", text="Amount")
        self.tree.heading("Description", text="Description")
        
        self.tree.column("Date", width=100)
        self.tree.column("Category", width=100)
        self.tree.column("Amount", width=80)
        self.tree.column("Description", width=200)
        
        scrollbar = ttk.Scrollbar(tree_frame, orient="vertical", command=self.tree.yview)
        self.tree.configure(yscrollcommand=scrollbar.set)
        
        self.tree.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        # Total Label
        self.total_label = tk.Label(self.root, text="Total Expenses: $0.00", font=("Arial", 14, "bold"), fg="#e74c3c")
        self.total_label.pack(pady=10)
    
    def add_expense(self):
        """Add expense to database"""
        date = self.date_entry.get_date().strftime("%Y-%m-%d")
        category = self.category_var.get()
        amount = self.amount_entry.get()
        description = self.desc_entry.get()
        
        if not category or not amount:
            messagebox.showwarning("Input Error", "Please fill in category and amount!")
            return
        
        try:
            amount = float(amount)
            
            self.cursor.execute("INSERT INTO expenses (date, category, amount, description) VALUES (?, ?, ?, ?)",
                              (date, category, amount, description))
            self.conn.commit()
            
            messagebox.showinfo("Success", "Expense added successfully!")
            self.clear_inputs()
            self.load_expenses()
        except ValueError:
            messagebox.showerror("Error", "Please enter a valid amount!")
    
    def load_expenses(self):
        """Load expenses from database"""
        for item in self.tree.get_children():
            self.tree.delete(item)
        
        self.cursor.execute("SELECT date, category, amount, description FROM expenses ORDER BY date DESC")
        expenses = self.cursor.fetchall()
        
        total = 0
        for expense in expenses:
            self.tree.insert("", "end", values=expense)
            total += expense[2]
        
        self.total_label.config(text=f"Total Expenses: ${total:.2f}")
    
    def clear_inputs(self):
        """Clear input fields"""
        self.date_entry.set_date(datetime.now())
        self.category_var.set("")
        self.amount_entry.delete(0, tk.END)
        self.desc_entry.delete(0, tk.END)
    
    def show_chart(self):
        """Seaborn - Show expense visualization"""
        self.cursor.execute("SELECT category, SUM(amount) FROM expenses GROUP BY category")
        data = self.cursor.fetchall()
        
        if not data:
            messagebox.showinfo("No Data", "No expenses to display!")
            return
        
        categories = [row[0] for row in data]
        amounts = [row[1] for row in data]
        
        # Set seaborn style
        sns.set_style("whitegrid")
        
        fig, ax = plt.subplots(figsize=(10, 6))
        sns.barplot(x=categories, y=amounts, palette="viridis", ax=ax)
        ax.set_title("Expenses by Category", fontsize=16, fontweight='bold')
        ax.set_xlabel("Category", fontsize=12)
        ax.set_ylabel("Amount ($)", fontsize=12)
        
        plt.tight_layout()
        plt.show()
    
    def predict_expense(self):
        """ML Libraries - Predict next month's expense using Linear Regression"""
        self.cursor.execute("SELECT date, amount FROM expenses ORDER BY date")
        data = self.cursor.fetchall()
        
        if len(data) < 5:
            messagebox.showinfo("Insufficient Data", "Need at least 5 expenses for prediction!")
            return
        
        # Prepare data for ML
        df = pd.DataFrame(data, columns=['date', 'amount'])
        df['date'] = pd.to_datetime(df['date'])
        df['days'] = (df['date'] - df['date'].min()).dt.days
        
        # Group by month and sum
        df['month'] = df['date'].dt.to_period('M')
        monthly = df.groupby('month')['amount'].sum().reset_index()
        monthly['month_num'] = range(len(monthly))
        
        
        # Train model
        X = monthly['month_num'].values.reshape(-1, 1)
        y = monthly['amount'].values
        
        model = LinearRegression()
        model.fit(X, y)
        
        # Predict next month
        next_month = len(monthly)
        prediction = model.predict([[next_month]])[0]
        
        messagebox.showinfo("Expense Prediction", 
                          f"Predicted expense for next month: ${prediction:.2f}\n\n"
                          f"Based on {len(monthly)} months of data")
    
    def export_csv(self):
        """File Handling - Export expenses to CSV file"""
        self.cursor.execute("SELECT date, category, amount, description FROM expenses")
        data = self.cursor.fetchall()
        
        if not data:
            messagebox.showinfo("No Data", "No expenses to export!")
            return
        
        filename = f"expenses_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        
        with open(filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Date', 'Category', 'Amount', 'Description'])
            writer.writerows(data)
        
        messagebox.showinfo("Success", f"Expenses exported to {filename}")
    
    def __del__(self):
        """Close database connection"""
        self.conn.close()

if __name__ == "__main__":
    root = tk.Tk()
    app = ExpenseTracker(root)
    root.mainloop()